In [27]:
import pandas as pd
import numpy as np
from tensorflow import keras

In [28]:
nb_rows = 100000
data = pd.read_csv("sudoku.csv", nrows=nb_rows)

print(f'{data.shape[0]} lignes et {data.shape[1]} colonnes')
data

100000 lignes et 2 colonnes


,puzzle,solution
0,0700000430400096108006349000940520003584600200...,6795182435437296188216349577943521863584617292...
1,3010865040465210705000000014008000020803479000...,3719865248465213795924738614638197522853479167...
2,0483015603600080909106700030200009355090102006...,7483915623652487919126754834217869355894132766...
3,0083170000042051090000400703271609049014500000...,2983176457642851391539462783271689549814537266...
4,0408906300001368208007405190004670524500207002...,1428956379751368248367425193984671524513287962...
...,...,...
99995,3257040900060037848470190021039028064891000036...,3257846919162537848476193521539728464891652736...
99996,0906340706050094007840006939060028045180062302...,1926345786357894217845216939763528145184762392...
99997,4050012000280070346300040980147609033001400858...,4953812671286975346372541985147689233691427858...
99998,0502963000900502800423019750615408374301780007...,8572963141934572866423819752615498374351786927...


In [29]:
for i in range(81):
    data[i] = data['puzzle'].str[i].astype('int32')
    
X = data.drop(['puzzle', 'solution'], axis=1)
X = np.array(X).reshape(X.shape[0], 9, 9, 1)

#X = (X/9)-0.5

print(X.shape)
X

(100000, 9, 9, 1)


array([[[[0],
         [7],
         [0],
         ...,
         [0],
         [4],
         [3]],

        [[0],
         [4],
         [0],
         ...,
         [6],
         [1],
         [0]],

        [[8],
         [0],
         [0],
         ...,
         [9],
         [0],
         [0]],

        ...,

        [[0],
         [8],
         [0],
         ...,
         [0],
         [9],
         [1]],

        [[9],
         [0],
         [2],
         ...,
         [0],
         [0],
         [5]],

        [[0],
         [0],
         [7],
         ...,
         [8],
         [0],
         [2]]],


       [[[3],
         [0],
         [1],
         ...,
         [5],
         [0],
         [4]],

        [[0],
         [4],
         [6],
         ...,
         [0],
         [7],
         [0]],

        [[5],
         [0],
         [0],
         ...,
         [0],
         [0],
         [1]],

        ...,

        [[0],
         [0],
         [4],
         ...,
         [2],


In [30]:
for i in range(81):
    data[i] = data['solution'].str[i].astype('int32')
    
y = data.drop(['puzzle', 'solution'], axis=1)
y = np.array(y).reshape(y.shape[0], 9, 9, 1)

#y = (y/9)-0.5
y = y-1
print(y.shape)
y

(100000, 9, 9, 1)


array([[[[5],
         [6],
         [8],
         ...,
         [1],
         [3],
         [2]],

        [[4],
         [3],
         [2],
         ...,
         [5],
         [0],
         [7]],

        [[7],
         [1],
         [0],
         ...,
         [8],
         [4],
         [6]],

        ...,

        [[3],
         [7],
         [4],
         ...,
         [2],
         [8],
         [0]],

        [[8],
         [5],
         [1],
         ...,
         [3],
         [6],
         [4]],

        [[0],
         [2],
         [6],
         ...,
         [7],
         [5],
         [1]]],


       [[[2],
         [6],
         [0],
         ...,
         [4],
         [1],
         [3]],

        [[7],
         [3],
         [5],
         ...,
         [2],
         [6],
         [8]],

        [[4],
         [8],
         [1],
         ...,
         [7],
         [5],
         [0]],

        ...,

        [[5],
         [2],
         [3],
         ...,
         [1],


In [22]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(81*9))
model.add(keras.layers.Reshape((81, 9)))
model.add(keras.layers.Activation('softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               patience=2, 
                                               restore_best_weights=True)

history = model.fit(X, y, epochs=20, validation_split=0.2, callbacks=[early_stopping]) #0.3

Train on 80000 samples, validate on 20000 samples
Epoch 1/20
80000/80000 [==============================] - 405s 5ms/sample - loss: 1.0050 - val_loss: 0.5395
Epoch 2/20
80000/80000 [==============================] - 406s 5ms/sample - loss: 0.4803 - val_loss: 0.4671
Epoch 3/20
80000/80000 [==============================] - 401s 5ms/sample - loss: 0.4160 - val_loss: 0.4584
Epoch 4/20
80000/80000 [==============================] - 384s 5ms/sample - loss: 0.3503 - val_loss: 0.4871
Epoch 5/20
80000/80000 [==============================] - 384s 5ms/sample - loss: 0.2641 - val_loss: 0.5623


In [13]:
pred = model.predict_classes(X[0].reshape(1,9,9,1)).reshape(9,9)+1
real = y[0].reshape(9,9)+1

In [14]:
pred == real

array([[ True,  True,  True, False, False,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True,  True, False,  True,  True],
       [False, False, False,  True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True,  True, False,  True,  True],
       [ True, False,  True,  True,  True,  True,  True, False,  True],
       [ True, False,  True, False,  True,  True,  True,  True,  True]])

In [15]:
pred

array([[6, 7, 9, 2, 8, 8, 2, 4, 3],
       [2, 4, 3, 7, 2, 9, 6, 1, 8],
       [8, 1, 1, 6, 3, 4, 9, 5, 7],
       [7, 9, 4, 3, 5, 2, 1, 8, 8],
       [3, 5, 8, 4, 6, 1, 1, 2, 9],
       [7, 2, 1, 8, 9, 7, 5, 3, 9],
       [4, 8, 5, 2, 7, 6, 4, 9, 1],
       [9, 3, 2, 1, 8, 3, 4, 6, 5],
       [1, 1, 7, 3, 4, 5, 8, 6, 2]], dtype=int64)

In [16]:
real

array([[6, 7, 9, 5, 1, 8, 2, 4, 3],
       [5, 4, 3, 7, 2, 9, 6, 1, 8],
       [8, 2, 1, 6, 3, 4, 9, 5, 7],
       [7, 9, 4, 3, 5, 2, 1, 8, 6],
       [3, 5, 8, 4, 6, 1, 7, 2, 9],
       [2, 1, 6, 8, 9, 7, 5, 3, 4],
       [4, 8, 5, 2, 7, 6, 3, 9, 1],
       [9, 6, 2, 1, 8, 3, 4, 7, 5],
       [1, 3, 7, 9, 4, 5, 8, 6, 2]])

In [17]:
acc = ((model.predict_classes(X).reshape(-1,9,9)+1 == y.reshape(-1,9,9)+1).sum())/(81*nb_rows)
print(f'{acc}% d\'exactitude') # 0.8452 (100 000) --> 0.8197 (1 000 000)

0.8385337037037037% d'exactitude
